# Multi-step LSTM Model (Encoder Decoder Model)

A model specifically developed for forecasting variable length output sequences is called the
Encoder-Decoder LSTM. The model was designed for prediction problems where there are
both input and output sequences, so-called sequence-to-sequence, or seq2seq problems, such
as translating text from one language to another. This model can be used for multi-step time
series forecasting. As its name suggests, the model is comprised of two sub-models: the encoder
and the decoder.

The encoder is a model responsible for reading and interpreting the input sequence. The
output of the encoder is a fixed length vector that represents the model's interpretation of the
sequence. The encoder is traditionally a Vanilla LSTM model, although other encoder models
can be used such as Stacked, Bidirectional, and CNN models.

The decoder uses the output of the encoder as an input. First, the fixed-length output of
the encoder is repeated, once for each required time step in the output sequence.

This sequence is then provided to an LSTM decoder model. The model must output a value
for each value in the output time step, which can be interpreted by a single output model.

In [1]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

Using TensorFlow backend.


#### split a univariate sequence into samples

In [2]:
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

## 1. Data Preparation

#### define input sequence

In [3]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

#### choose a number of time steps

In [4]:
n_steps_in, n_steps_out = 3, 2

#### convert into input/output

In [5]:
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

In [6]:
for i in range(len(X)):
	print(X[i], y[i])

[10 20 30] [40 50]
[20 30 40] [50 60]
[30 40 50] [60 70]
[40 50 60] [70 80]
[50 60 70] [80 90]


#### reshape from [samples, timesteps] into [samples, timesteps, features]

In [7]:
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
y = y.reshape((y.shape[0], y.shape[1], n_features))

## 2. LSTM

#### define model

In [8]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

#### fit model

In [9]:
model.fit(X, y, epochs=100, verbose=0)

#### demonstrate prediction

In [10]:
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[[102.07834]
  [113.33665]]]
